In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn import cross_validation
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn.model_selection import KFold, cross_val_score

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("glass.csv")
train.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [3]:
train=np.asarray(train)
X=np.zeros((214,9))
y=np.zeros((214,1))
X=np.asarray(X)
y=np.asarray(y)
train.shape

(428, 10)

In [4]:
k=0
sig=0.1
r=0.1
m=np.zeros((214,9))
m=np.asarray(m)

In [5]:
print(np.shape(m))

(214, 9)


In [6]:
 count=0

In [7]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
   # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

def gm(y_predict,y_test):
    test_min=0
    test_max=0
    pred_min=0
    pred_max=0
    y_test=np.asarray(y_test)
    if(y_test[0]==1):
         test_min=test_min+1
    else:
        test_max=test_max+1
    print("y_test min",test_min)       
    print("y_test max",test_max)
    if(y_predict[0]==1 and y_predict[0]==y_test[0]):
         pred_min=pred_min+1
    elif(y_predict[0]==-1 and y_predict[0]==y_test[0]):
         pred_max=pred_max+1
    print("y_pred min",pred_min)       
    print("y_pred max",pred_max)

In [8]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y,m):
        self.kernel = gaussian_kernel
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = gaussian_kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None
        m=np.asarray(m)
        self.K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K1[i][j] = gaussian_kernel(m[i], X[j])
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [9]:

if __name__ == "__main__":
    import pylab as pl  
   
    def test_soft():
        count=0
        for v in range (0,214):
            for i in range (0,214):
                for j in range(9):
                    X[i][j]=train[i+v][j]
                    y[i]=train[i+v][9]
                for j in range(9):
                    m[i][j]=((X[i][j]-(k*r*sig))+(X[i][j]+(k*(2-r)*sig)))/2
                    #rint(m[i][j])
            y_train=np.zeros((213))
            y_test=np.zeros((1))
            for i in range(0,213):
                y_train[i]=y[i]
            y_test[0]=y[213]
            X_train=np.zeros((213,9))
            X_test=np.zeros((1,9))
            for i in range(0,213):
                for j in range(0,9):
                    X_train[i][j]=X[i][j]
            for j in range (0,9):
                X_test[0][j]=X[213][j]

            clf = SVM(C=100.0)
            clf.fit(X_train, y_train,m)

            y_predict = clf.predict(X_test)
            gm(y_predict,y_test)
            correct = np.sum(y_predict == y_test)
            if(correct==1):
                count=count+1
            print("%d out of %d predictions correct" % (correct, len(y_predict)))
            print(correct/len(y_predict))
            print(count)
        print("%d no of misclassification"%(214-count))
        print("Probability of misclassification",(214-count)/214)

    test_soft()
    #test_non_linear()
    

(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3885e+04 -9.4732e+05  1e+06  1e-01  4e-13
 1:  5.7785e+03 -1.0954e+05  1e+05  1e-02  5e-13
 2: -4.5492e+03 -2.3146e+04  2e+04  1e-03  5e-13
 3: -7.3556e+03 -1.3984e+04  7e+03  3e-04  5e-13
 4: -8.4694e+03 -1.2253e+04  4e+03  1e-04  6e-13
 5: -9.2169e+03 -1.1130e+04  2e+03  6e-05  7e-13
 6: -9.7233e+03 -1.0329e+04  6e+02  2e-05  7e-13
 7: -9.9066e+03 -1.0089e+04  2e+02  3e-06  8e-13
 8: -9.9676e+03 -1.0010e+04  4e+01  6e-07  7e-13
 9: -9.9850e+03 -9.9881e+03  3e+00  1e-08  9e-13
10: -9.9863e+03 -9.9866e+03  3e-01  9e-10  8e-13
11: -9.9865e+03 -9.9865e+03  9e-03  3e-11  9e-13
Optimal solution found.
optimal
(213,)
157 support vectors out of 213 points
y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
1
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3532e+04 -9.5222e+05  1e+06  1e-01  4e-13
 1:  6.0150e+03 -1.1136e+05  1e+05  1e-02  5e-13
 2: -4.4197e+03 -2.3357e+04  2

(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3798e+04 -9.3981e+05  1e+06  1e-01  4e-13
 1:  5.7876e+03 -1.0946e+05  1e+05  1e-02  5e-13
 2: -4.4850e+03 -2.3225e+04  2e+04  1e-03  4e-13
 3: -7.2637e+03 -1.3951e+04  7e+03  3e-04  5e-13
 4: -8.3787e+03 -1.2173e+04  4e+03  1e-04  5e-13
 5: -9.1216e+03 -1.1044e+04  2e+03  6e-05  6e-13
 6: -9.5915e+03 -1.0296e+04  7e+02  2e-05  6e-13
 7: -9.7934e+03 -1.0027e+04  2e+02  4e-06  7e-13
 8: -9.8805e+03 -9.9141e+03  3e+01  5e-07  7e-13
 9: -9.8922e+03 -9.8993e+03  7e+00  7e-08  7e-13
10: -9.8952e+03 -9.8957e+03  5e-01  4e-09  7e-13
11: -9.8954e+03 -9.8954e+03  8e-03  7e-11  8e-13
Optimal solution found.
optimal
(213,)
155 support vectors out of 213 points
y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
4
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.4111e+04 -9.4291e+05  1e+06  1e-01  4e-13
 1:  6.1478e+03 -1.1039e+05  1e+05  1e-02  5e-13
 2: -4.3028e+03 -2.3445e+04  2

y_test min 1
y_test max 0
y_pred min 0
y_pred max 0
0 out of 1 predictions correct
0.0
11
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3924e+04 -9.4455e+05  1e+06  1e-01  4e-13
 1:  6.0341e+03 -1.1080e+05  1e+05  1e-02  4e-13
 2: -4.3732e+03 -2.3387e+04  2e+04  1e-03  4e-13
 3: -7.1810e+03 -1.3887e+04  7e+03  3e-04  4e-13
 4: -8.2473e+03 -1.2155e+04  4e+03  2e-04  5e-13
 5: -9.0281e+03 -1.0935e+04  2e+03  6e-05  5e-13
 6: -9.5103e+03 -1.0183e+04  7e+02  2e-05  6e-13
 7: -9.7111e+03 -9.9152e+03  2e+02  4e-06  6e-13
 8: -9.7927e+03 -9.8121e+03  2e+01  2e-07  7e-13
 9: -9.7998e+03 -9.8033e+03  4e+00  2e-08  7e-13
10: -9.8013e+03 -9.8015e+03  3e-01  2e-09  6e-13
11: -9.8014e+03 -9.8014e+03  6e-03  4e-11  6e-13
Optimal solution found.
optimal
(213,)
153 support vectors out of 213 points
y_test min 1
y_test max 0
y_pred min 0
y_pred max 0
0 out of 1 predictions correct
0.0
11
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3219e+04 -9.3617e+05  1e+06  1

y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
19
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3612e+04 -9.3912e+05  1e+06  1e-01  3e-13
 1:  5.6359e+03 -1.0967e+05  1e+05  1e-02  4e-13
 2: -4.5270e+03 -2.3255e+04  2e+04  1e-03  4e-13
 3: -7.3290e+03 -1.3988e+04  7e+03  3e-04  4e-13
 4: -8.4566e+03 -1.2217e+04  4e+03  1e-04  4e-13
 5: -9.1929e+03 -1.1107e+04  2e+03  6e-05  5e-13
 6: -9.6653e+03 -1.0348e+04  7e+02  2e-05  6e-13
 7: -9.8665e+03 -1.0081e+04  2e+02  4e-06  6e-13
 8: -9.9464e+03 -9.9769e+03  3e+01  4e-07  6e-13
 9: -9.9578e+03 -9.9626e+03  5e+00  6e-08  5e-13
10: -9.9598e+03 -9.9601e+03  3e-01  4e-09  6e-13
11: -9.9599e+03 -9.9599e+03  6e-03  7e-11  6e-13
Optimal solution found.
optimal
(213,)
153 support vectors out of 213 points
y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
20
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3524e+04 -9.3732e+05  1e+06  1

(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3496e+04 -9.3806e+05  1e+06  1e-01  3e-13
 1:  5.5372e+03 -1.0934e+05  1e+05  1e-02  4e-13
 2: -4.5632e+03 -2.3212e+04  2e+04  1e-03  4e-13
 3: -7.3616e+03 -1.3969e+04  7e+03  3e-04  4e-13
 4: -8.4824e+03 -1.2239e+04  4e+03  1e-04  4e-13
 5: -9.2233e+03 -1.1126e+04  2e+03  6e-05  5e-13
 6: -9.7292e+03 -1.0322e+04  6e+02  2e-05  5e-13
 7: -9.9091e+03 -1.0086e+04  2e+02  3e-06  6e-13
 8: -9.9681e+03 -1.0009e+04  4e+01  6e-07  6e-13
 9: -9.9851e+03 -9.9880e+03  3e+00  9e-09  6e-13
10: -9.9863e+03 -9.9866e+03  3e-01  8e-10  6e-13
11: -9.9865e+03 -9.9865e+03  9e-03  2e-11  6e-13
Optimal solution found.
optimal
(213,)
155 support vectors out of 213 points
y_test min 1
y_test max 0
y_pred min 1
y_pred max 0
1 out of 1 predictions correct
1.0
29
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3564e+04 -9.3833e+05  1e+06  1e-01  3e-13
 1:  5.5798e+03 -1.0920e+05  1e+05  1e-02  4e-13
 2: -4.5525e+03 -2.3235e+04  

...

y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
126
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3772e+04 -9.4827e+05  1e+06  1e-01  3e-13
 1:  5.7594e+03 -1.0931e+05  1e+05  1e-02  4e-13
 2: -4.5448e+03 -2.3088e+04  2e+04  1e-03  4e-13
 3: -7.3559e+03 -1.3950e+04  7e+03  3e-04  4e-13
 4: -8.4604e+03 -1.2241e+04  4e+03  2e-04  5e-13
 5: -9.2216e+03 -1.1113e+04  2e+03  6e-05  5e-13
 6: -9.7201e+03 -1.0331e+04  6e+02  2e-05  6e-13
 7: -9.9057e+03 -1.0089e+04  2e+02  3e-06  6e-13
 8: -9.9674e+03 -1.0010e+04  4e+01  6e-07  6e-13
 9: -9.9850e+03 -9.9882e+03  3e+00  1e-08  7e-13
10: -9.9863e+03 -9.9866e+03  3e-01  1e-09  7e-13
11: -9.9865e+03 -9.9865e+03  9e-03  3e-11  7e-13
Optimal solution found.
optimal
(213,)
157 support vectors out of 213 points
y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
127
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3980e+04 -9.4836e+05  1e+06 

(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3683e+04 -9.4348e+05  1e+06  1e-01  4e-13
 1:  5.6320e+03 -1.0889e+05  1e+05  1e-02  4e-13
 2: -4.5762e+03 -2.3031e+04  2e+04  1e-03  5e-13
 3: -7.3718e+03 -1.3943e+04  7e+03  3e-04  5e-13
 4: -8.4706e+03 -1.2234e+04  4e+03  1e-04  6e-13
 5: -9.2201e+03 -1.1118e+04  2e+03  6e-05  6e-13
 6: -9.7228e+03 -1.0328e+04  6e+02  2e-05  6e-13
 7: -9.9060e+03 -1.0089e+04  2e+02  3e-06  7e-13
 8: -9.9675e+03 -1.0010e+04  4e+01  6e-07  6e-13
 9: -9.9850e+03 -9.9882e+03  3e+00  1e-08  7e-13
10: -9.9863e+03 -9.9866e+03  3e-01  1e-09  8e-13
11: -9.9865e+03 -9.9865e+03  9e-03  3e-11  8e-13
Optimal solution found.
optimal
(213,)
158 support vectors out of 213 points
y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
137
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3835e+04 -9.4935e+05  1e+06  1e-01  4e-13
 1:  5.8103e+03 -1.0976e+05  1e+05  1e-02  5e-13
 2: -4.5434e+03 -2.3184e+04 

y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
146
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3720e+04 -9.4627e+05  1e+06  1e-01  4e-13
 1:  5.7179e+03 -1.0926e+05  1e+05  1e-02  4e-13
 2: -4.5613e+03 -2.3112e+04  2e+04  1e-03  5e-13
 3: -7.3638e+03 -1.3975e+04  7e+03  3e-04  5e-13
 4: -8.4737e+03 -1.2245e+04  4e+03  1e-04  6e-13
 5: -9.2170e+03 -1.1129e+04  2e+03  6e-05  6e-13
 6: -9.7254e+03 -1.0326e+04  6e+02  2e-05  7e-13
 7: -9.9069e+03 -1.0088e+04  2e+02  3e-06  9e-13
 8: -9.9677e+03 -1.0010e+04  4e+01  6e-07  8e-13
 9: -9.9851e+03 -9.9881e+03  3e+00  1e-08  8e-13
10: -9.9863e+03 -9.9866e+03  3e-01  9e-10  8e-13
11: -9.9865e+03 -9.9865e+03  9e-03  3e-11  8e-13
Optimal solution found.
optimal
(213,)
156 support vectors out of 213 points
y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
147
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3794e+04 -9.4579e+05  1e+06 

y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
156
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3917e+04 -9.4888e+05  1e+06  1e-01  4e-13
 1:  5.8171e+03 -1.0969e+05  1e+05  1e-02  5e-13
 2: -4.5410e+03 -2.3178e+04  2e+04  1e-03  5e-13
 3: -7.3518e+03 -1.3989e+04  7e+03  3e-04  5e-13
 4: -8.4673e+03 -1.2258e+04  4e+03  1e-04  6e-13
 5: -9.2174e+03 -1.1130e+04  2e+03  6e-05  7e-13
 6: -9.7232e+03 -1.0330e+04  6e+02  2e-05  7e-13
 7: -9.9065e+03 -1.0089e+04  2e+02  3e-06  7e-13
 8: -9.9676e+03 -1.0010e+04  4e+01  6e-07  7e-13
 9: -9.9850e+03 -9.9881e+03  3e+00  1e-08  8e-13
10: -9.9863e+03 -9.9866e+03  3e-01  9e-10  7e-13
11: -9.9865e+03 -9.9865e+03  9e-03  3e-11  9e-13
Optimal solution found.
optimal
(213,)
157 support vectors out of 213 points
y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
157
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3913e+04 -9.4855e+05  1e+06 

(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3885e+04 -9.4730e+05  1e+06  1e-01  3e-13
 1:  5.7790e+03 -1.0957e+05  1e+05  1e-02  5e-13
 2: -4.5486e+03 -2.3149e+04  2e+04  1e-03  5e-13
 3: -7.3553e+03 -1.3982e+04  7e+03  3e-04  5e-13
 4: -8.4690e+03 -1.2252e+04  4e+03  1e-04  5e-13
 5: -9.2158e+03 -1.1131e+04  2e+03  6e-05  6e-13
 6: -9.7237e+03 -1.0329e+04  6e+02  2e-05  7e-13
 7: -9.9066e+03 -1.0089e+04  2e+02  3e-06  7e-13
 8: -9.9676e+03 -1.0010e+04  4e+01  6e-07  8e-13
 9: -9.9851e+03 -9.9881e+03  3e+00  1e-08  8e-13
10: -9.9863e+03 -9.9866e+03  3e-01  9e-10  8e-13
11: -9.9865e+03 -9.9865e+03  9e-03  3e-11  8e-13
Optimal solution found.
optimal
(213,)
157 support vectors out of 213 points
y_test min 0
y_test max 1
y_pred min 0
y_pred max 1
1 out of 1 predictions correct
1.0
167
(213, 213)
     pcost       dcost       gap    pres   dres
 0:  1.3891e+04 -9.5222e+05  1e+06  1e-01  4e-13
 1:  5.8992e+03 -1.1010e+05  1e+05  1e-02  5e-13
 2: -4.5223e+03 -2.3252e+04 